# Linear dos mínimos quadrados(Cap10) - Pense Estatística, 2º Edição.

## Universidade Federal de Alagoas - UFAL
## Centro de Tecnologia - CTEC
## Programa de Pós-Graduação Recursos Hídricos e Saneamento - PPGRHS
### Disciplina de Estatística

Clebson Farias

In [2]:
from __future__ import print_function, division

%matplotlib inline

import numpy as np
import pandas as pd

import thinkstats2
import thinkplot

## Ajuste dos mínimos quadrados
- Os coeficientes de correlação medem a força e o sinal de uma relação, mas não a inclinação;
- A forma mais comum para estimar a inclinação é o ajuste de mínimos quadrados;
- **Ajuste linear** é uma linha destinada a modelar a relação entre variáveis;
- O ajuste de **mínimos quadrados** minimiza o erro quadrático médio (MSE) entre a linha e os dados;
- ys é expressar em função de outra sequência xs, onde:
\begin{equation}
    ys[i] = inter + slope * xs[i] + \epsilon
\end{equation}
- $\epsilon$ É um erro aleatório de média 0 e variância $\sigma^{2}$
- Sendo assim, o valor esperado de ys pertence a uma distribuição normal com parâmetros $\mu$, sendo:
\begin{equation}
    \mu_{ys|x} = inter + slope * xs[i]
\end{equation}
- Esta previsão é apenas aproximada, o desvio residual é,
\begin{equation}
    res = ys - (inter + slope * xs)
\end{equation}
- Os **resíduos** são devidos a **fatores aleatórios** como erro de medição ou **fatores não aleatórios** que são desconhecidos;
- O objetivo dos parâmetros inter e slope, é diminuir o resíduo, isso pode ser feito de várias formas:
    - Minimizar o valor absoluto;
    - Seus quadrados;
    - Seus cubos;
    - **Minimizar a soma dos resíduos ao quadrado**.
- Logo abaixo está listado as razões para minimizar a soma dos resíduos ao quadrado:
    - Trata os resíduos positivos e negativos da mesma forma;
    - Dá mais peso aos grandes resíduos;
    - É um estimador de máxima verosimilhança de inter e slope;
    - Os valores de inter e slope podem ser calculados de forma eficiente.

## Implementação

In [3]:
#Dados Manso
dados = pd.read_csv("manso.csv", index_col=0, parse_dates=True)
dados

,1455008,66210000,66231000,MANSO
1931-01-01,NaN,NaN,NaN,NaN
1931-01-02,NaN,NaN,NaN,NaN
1931-01-03,NaN,NaN,NaN,NaN
1931-01-04,NaN,NaN,NaN,NaN
1931-01-05,NaN,NaN,NaN,NaN
1931-01-06,NaN,NaN,NaN,NaN
1931-01-07,NaN,NaN,NaN,NaN
1931-01-08,NaN,NaN,NaN,NaN
1931-01-09,NaN,NaN,NaN,NaN
1931-01-10,NaN,NaN,NaN,NaN
